# Analysing food hygiene data

This notebook contains the code for analysing food hygiene data to establish the scale and nature of uninspected establishments.

We need to:

* Compile: fetch data from the FSA
* Clean: convert these from XML files to dataframes
* Question: find out how many haven't been inspected in 2 years or more (or other timeframes)
* Question: find out how many haven't yet been inspected
* Context: work this out as a percentage
* Context: establish the makeup of those establishments (e.g. how many are rated below 3? How many just 3?)
* Combine: repeat this for all authorities
* Combine: fetch Google Places API data
* Context: what's the average rating of those places not inspected?

## Import the libraries

In [ ]:
#import the libraries we'll need
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup  # Change from ET to BeautifulSoup
import pandas as pd

In [ ]:
#import a library for downloading files
from google.colab import files

In [ ]:
#for using the isnan() function to check for missing values
import math

## Fetch the XML file

Each XML file is at a different URL on https://ratings.food.gov.uk/open-data - we try one.

In [ ]:
#Brentwood's FSA data URL
url = "https://ratings.food.gov.uk/api/open-data-files/FHRS111en-GB.xml"

# Fetch the URL
response = requests.get(url)
# Store the content
xml_data = response.content

## Parse into a 'soup' and then into a dataframe

Now that we have a requests object, we need to convert that to a BeautifulSoup object to be able to parse it as a structured set of info.

As we parse it we store the info in a `pandas` dataframe.

In [ ]:
# Parse the XML data
soup = BeautifulSoup(xml_data, 'xml')  # Use BeautifulSoup

# Create empty lists to store data
establishments = []
business_names = []
address_line_1s = []
address_line_2s = []
address_line_3s = []
address_line_4s = []
post_codes = []
rating_values = []
rating_dates = []
business_types = []
las = []
nrps = []
lats = []
lngs = []

# Find all establishment details
establishments_data = soup.find_all('EstablishmentDetail')

# Extract data for each establishment
for establishment in establishments_data:
  business_names.append(establishment.find('BusinessName').text.strip() or "")
  las.append(establishment.find('LocalAuthorityName').text.strip() or "")
  nrps.append(establishment.find('NewRatingPending').text.strip() or "")
  #if it is there
  if establishment.find('AddressLine1') != None:
    address_line_1s.append(establishment.find('AddressLine1').text.strip() or "")
  else:
    address_line_1s.append('')
  if establishment.find('AddressLine2') != None:
    address_line_2s.append(establishment.find('AddressLine2').text.strip() or "")
  else:
    address_line_2s.append('')
  if establishment.find('AddressLine3') != None:
    address_line_3s.append(establishment.find('AddressLine3').text.strip() or "")
  else:
    address_line_3s.append('')
  if establishment.find('AddressLine4') != None:
    address_line_4s.append(establishment.find('AddressLine4').text.strip() or "")
  else:
    address_line_4s.append('')
  if establishment.find('PostCode') != None:
    post_codes.append(establishment.find('PostCode').text.strip() or "")
  else:
    post_codes.append('')
  rating_values.append(establishment.find('RatingValue').text.strip() or "")
  rating_dates.append(establishment.find('RatingDate').text.strip() or "")
  business_types.append(establishment.find('BusinessType').text.strip() or "")

  # Find Geocode data (might not exist)
  #if establishment.find('Geocode') != None:
  #print(establishment.find('Geocode'))
  geocode = establishment.find('Geocode')
  #print(geocode)
  if geocode.find('Latitude') != None:
    lats.append(geocode.find('Latitude').text.strip())
    lngs.append(geocode.find('Longitude').text.strip())
  else:
    lats.append("")
    lngs.append("")

# Create a dictionary from lists
data = {
    "BusinessName": business_names,
    "Authority": las,
    "AddressLine1": address_line_1s,
    "AddressLine2": address_line_2s,
    "AddressLine3": address_line_3s,
    "AddressLine4": address_line_4s,
    "PostCode": post_codes,
    "RatingValue": rating_values,
    "NewRatingPending": nrps,
    "RatingDate": rating_dates,
    "BusinessType": business_types,
    "Lat": lats,
    "Lng": lngs
}

# Create pandas dataframe
df = pd.DataFrame(data)

# Print
df


,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng
0,:ROSEBANK NURSING HOMES LTD T/A Ardtully Retir...,Brentwood,Ardtully Retirement Home,Station Lane,Ingatestone,Essex,CM4 0BL,5,False,2023-11-20,Hospitals/Childcare/Caring Premises,51.666644,0.380843
1,124 (Essex) Transport Squadron Rlc Army Reserv...,Brentwood,Territorial Army Centre,Clive Road,Warley,Brentwood,CM13 3UJ,5,False,2023-07-31,Pub/bar/nightclub,51.599399,0.296689
2,55 Above Ltd,Brentwood,,,,,,5,False,2020-01-21,Retailers - other,,
3,A & S,Brentwood,13 Eastham Crescent,Brentwood,Essex,,CM13 2BN,5,False,2023-02-14,Retailers - other,51.612116,0.327679
4,A B Roots,Brentwood,Spring Farm,Blackmore Road,Ingatestone,Essex,CM4 0NP,5,False,2019-07-15,Retailers - other,51.677724,0.355835
...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,Yiamas \& NYX,Brentwood,Restaurant,Yiamas,Ongar Road,Pilgrims Hatch,CM15 9SS,5,False,2023-11-17,Other catering premises,51.6544384,0.2683294
661,Yorkies,Brentwood,186 Warley Hill,Warley,Essex,,CM14 5HF,5,False,2023-02-13,Restaurant/Cafe/Canteen,51.6057156,0.2965792
662,ZEBRANO,Brentwood,161 Kings Road,Brentwood,Essex,,CM14 4EG,5,False,2023-12-06,Other catering premises,51.6150512695313,0.299198001623154
663,Zizzi,Brentwood,72-74 High Street,Brentwood,Essex,,CM14 4AN,5,False,2022-06-17,Other catering premises,51.619841,0.301203


### Identify the business types

We are not going to look at all businesses, so we need a list of types that we might choose from.

In [ ]:
#show the unique values, and the count of each
df['BusinessType'].value_counts()

Restaurant/Cafe/Canteen                  122
Other catering premises                  121
Retailers - other                        116
Pub/bar/nightclub                         89
Takeaway/sandwich shop                    52
School/college/university                 49
Hospitals/Childcare/Caring Premises       44
Mobile caterer                            36
Retailers - supermarkets/hypermarkets     18
Manufacturers/packers                      8
Hotel/bed & breakfast/guest house          5
Importers/Exporters                        2
Distributors/Transporters                  2
Farmers/growers                            1
Name: BusinessType, dtype: int64

### Filtering to select categories

The categories we are going to go with as fitting into our criteria of 'places a person might eat out' are:

* Restaurant/Cafe/Canteen
* Pub/bar/nightclub
* Takeaway/sandwich shop
* Mobile caterer


We are going to exclude 'Other catering premises' as inspection suggests this is almost entirely sports clubs and home-based cake/baking operations.

In [ ]:
filtereddf = df[
    (df['BusinessType'] == 'Restaurant/Cafe/Canteen') |
    (df['BusinessType'] == 'Pub/bar/nightclub') |
    (df['BusinessType'] == 'Takeaway/sandwich shop') |
    (df['BusinessType'] == 'Mobile caterer')
 ]

filtereddf['BusinessType'].value_counts()

Restaurant/Cafe/Canteen    122
Pub/bar/nightclub           89
Takeaway/sandwich shop      52
Mobile caterer              36
Name: BusinessType, dtype: int64

### FUNCTION: parsefsaxml

We are going to need to do this repeatedly, so let's store in a function.

In [ ]:
#define the function - it takes one parameter we call 'url'
def parsefsaxml(url):
  # Fetch the URL
  response = requests.get(url)
  # Store the content
  xml_data = response.content
  # Parse the XML data
  soup = BeautifulSoup(xml_data, 'xml')
  # Create empty lists to store data
  establishments = []
  business_names = []
  address_line_1s = []
  address_line_2s = []
  address_line_3s = []
  address_line_4s = []
  post_codes = []
  rating_values = []
  rating_dates = []
  business_types = []
  las = []
  nrps = []
  lats = []
  lngs = []

  # Find all establishment details
  establishments_data = soup.find_all('EstablishmentDetail')

  # Extract data for each establishment
  for establishment in establishments_data:
    business_names.append(establishment.find('BusinessName').text.strip() or "")
    las.append(establishment.find('LocalAuthorityName').text.strip() or "")
    nrps.append(establishment.find('NewRatingPending').text.strip() or "")
    #if it is there
    if establishment.find('AddressLine1') != None:
      address_line_1s.append(establishment.find('AddressLine1').text.strip() or "")
    else:
      address_line_1s.append('')
    if establishment.find('AddressLine2') != None:
      address_line_2s.append(establishment.find('AddressLine2').text.strip() or "")
    else:
      address_line_2s.append('')
    if establishment.find('AddressLine3') != None:
      address_line_3s.append(establishment.find('AddressLine3').text.strip() or "")
    else:
      address_line_3s.append('')
    if establishment.find('AddressLine4') != None:
      address_line_4s.append(establishment.find('AddressLine4').text.strip() or "")
    else:
      address_line_4s.append('')
    if establishment.find('PostCode') != None:
      post_codes.append(establishment.find('PostCode').text.strip() or "")
    else:
      post_codes.append('')
    #this trips up on https://ratings.food.gov.uk/api/open-data-files/FHRS527en-GB.xml
    if establishment.find('RatingValue') != None:
      rating_values.append(establishment.find('RatingValue').text.strip() or "")
    else:
      rating_values.append('')
    rating_dates.append(establishment.find('RatingDate').text.strip() or "")
    business_types.append(establishment.find('BusinessType').text.strip() or "")

    # Find Geocode data (might not exist)
    #if establishment.find('Geocode') != None:
    #print(establishment.find('Geocode'))
    geocode = establishment.find('Geocode')
    #print(geocode)
    if geocode.find('Latitude') != None:
      lats.append(geocode.find('Latitude').text.strip())
      lngs.append(geocode.find('Longitude').text.strip())
    else:
      lats.append("")
      lngs.append("")

  # Create a dictionary from lists
  data = {
      "BusinessName": business_names,
      "Authority": las,
      "AddressLine1": address_line_1s,
      "AddressLine2": address_line_2s,
      "AddressLine3": address_line_3s,
      "AddressLine4": address_line_4s,
      "PostCode": post_codes,
      "RatingValue": rating_values,
      "NewRatingPending": nrps,
      "RatingDate": rating_dates,
      "BusinessType": business_types,
      "Lat": lats,
      "Lng": lngs
  }
  # Create pandas dataframe
  df = pd.DataFrame(data)
  #return to whatever called the function
  return(df)


### Extract the year separately

The `RatingDate` column is currently a text string. As we want to filter on year, we can extract that into a dedicated column.

In [ ]:
#use .to_datetime() from pandas to convert the column to datetime
#add the method .dt.year to extract the year from the resulting list of datetime objects
df['ratingYear'] = pd.to_datetime(df['RatingDate']).dt.year

## Filter to those inspected before 2019

In [ ]:
#
before2019 = df[df['ratingYear'] < 2019]
before2019

,BusinessName,AddressLine1,AddressLine2,PostCode,RatingValue,RatingDate,BusinessType,Lat,Lng,ratingYear
9,A.S.K. Wines,88 Church Lane,Doddinghurst,CM15 0NG,Exempt,2018-02-12,Retailers - other,51.667835,0.298024,2018.0
15,Adele Bywater Cakes,,,,5,2016-01-25,Other catering premises,,,2016.0
35,Bar Bar.Co,159 Kings Road,Brentwood,CM14 4EG,5,2017-11-13,Retailers - other,51.6150512695313,0.299198001623154,2017.0
44,Bentley District Village Club,Bentley Village Hall,Ongar Road,CM15 9RZ,5,2014-10-21,Pub/bar/nightclub,51.640834,0.275657,2014.0
59,Boots UK Ltd,51 High Street,Brentwood,CM14 4RH,5,2017-07-03,Retailers - other,51.620859,0.302945,2017.0
...,...,...,...,...,...,...,...,...,...,...
626,Travis Perkins,41 Coxtie Green Road,Pilgrims Hatch,CM14 5PN,5,2017-12-08,Retailers - other,51.639362,0.27502,2017.0
629,Vaporetto,,,,5,2015-10-07,Mobile caterer,,,2015.0
636,W H Smith Ltd,1 - 2 Baytree Centre,Brentwood,CM14 4BX,Exempt,2015-07-06,Retailers - other,51.619507,0.302212,2015.0
643,Well Ltd,201 Rayleigh Road,Hutton,CM13 1LZ,Exempt,2017-12-27,Retailers - other,51.6329536437988,0.351101011037827,2017.0


### Show that as a percentage

We can see what that is as a percentage by dividing the length (number of rows) of the filtered dataset by the length of the unfiltered dataset.

In [ ]:
len(before2019)/len(df)

0.13922155688622753

## What are these establishments like?

We use `.value_counts()` to generate a pivot table of how many rows there are in each category (rating value).

In [ ]:
before2019['RatingValue'].value_counts()

5         71
Exempt    17
4          5
Name: RatingValue, dtype: int64

In [ ]:
#divide all by the rows to get as %
before2019['RatingValue'].value_counts()/len(before2019)

5         0.763441
Exempt    0.182796
4         0.053763
Name: RatingValue, dtype: float64

## Repeat for 'Awaiting inspection'

Some establishments don't have any date because they are 'Awaiting inspection'. Let's look at them:

In [ ]:
len(df[df['RatingValue'] == 'AwaitingInspection' ])

17

In [ ]:
#divide the part by the whole
len(df[df['RatingValue'] == 'AwaitingInspection' ])/len(df)

0.025449101796407185

In [ ]:
#add the two together
ai_perc = len(df[df['RatingValue'] == 'AwaitingInspection' ])/len(df)
pre19perc = len(before2019)/len(df)
ai_perc+pre19perc

0.16467065868263472

## Fetch the codes for each authority

We have collected the codes for each authority covered by the FSA, and stored them in a Google Doc, which is imported below.

Because the ID codes are numeric, they will be imported as numbers unless we specify otherwise, so we add the `dtype=str` parameter below to ensure all data is imported as strings.

In [ ]:
#store the URL we've published the Google Sheet at (as a CSV)
fsacodesurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vT56kwmL6BGdve2HLvPqazY9qIOC9R9OC6-yzcmwnaKgca3MrImKe2-tPF7ltlE29OkPn9ioiSBuDSi/pub?gid=1903756103&single=true&output=csv"
#import, all fields as strings
fsacodedf = pd.read_csv(fsacodesurl, dtype=str)
#show
fsacodedf

,ID,LA only
0,297,Babergh
1,109,Basildon
2,701,Bedford
3,110,Braintree
4,227,Breckland
...,...,...
358,567,Rhondda Cynon Taf
359,568,Swansea
360,569,Torfaen
361,570,Vale of Glamorgan


### Generate a list of URLs

These codes mean we can now generate URLs for the API endpoint for each authority.

The URLs look like this:

`https://ratings.food.gov.uk/api/open-data-files/FHRS561en-GB.xml`

The only bit that changes is the three-digit code after `FHRS`.

In [ ]:
#create a list to store the urls
apiurl_list = []
#loop through the codes
for i in fsacodedf['ID']:
  #form the URL with that in the middle
  apiurl = "https://ratings.food.gov.uk/api/open-data-files/FHRS"+i+"en-GB.xml"
  #append to the list
  apiurl_list.append(apiurl)

#show the first 5 results
apiurl_list[:5]

['https://ratings.food.gov.uk/api/open-data-files/FHRS297en-GB.xml',
 'https://ratings.food.gov.uk/api/open-data-files/FHRS109en-GB.xml',
 'https://ratings.food.gov.uk/api/open-data-files/FHRS701en-GB.xml',
 'https://ratings.food.gov.uk/api/open-data-files/FHRS110en-GB.xml',
 'https://ratings.food.gov.uk/api/open-data-files/FHRS227en-GB.xml']

### Loop through in groups of 100 - filtering by business type

Now we test our function on multiple XML files from the API.

We start by testing 5 by adding an index slice in:

`for i in apiurl_list[:5]:`

Then we change it to `[:100]`, then to `[100:200]` and finally `[200:]`, each time storing the results in a different dataframe so we can recombine it later.

This helps us deal with problems - at one point an empty rating cell trips up the process so this limits the impact to just one third of the total.

#### The first 100

In [ ]:
#create an empty list to store the results
df_list = []

#loop through api URLs
for i in apiurl_list[:100]:
  print(i)
  idf = parsefsaxml(i)
  #filter out exempt inspections
  idf = idf[idf['RatingValue'] != 'Exempt']
  #filter to the eating out categories
  idf = idf[
    (idf['BusinessType'] == 'Restaurant/Cafe/Canteen') |
    (idf['BusinessType'] == 'Pub/bar/nightclub') |
    (idf['BusinessType'] == 'Takeaway/sandwich shop') |
    (idf['BusinessType'] == 'Mobile caterer')
  ]
  # Append the new DataFrame to the list
  df_list.append(idf)

# Concatenate all DataFrames in the list into a single DataFrame
alldf = pd.concat(df_list)

alldf

#store this 100 in one data frame
alldf0_99 = alldf

https://ratings.food.gov.uk/api/open-data-files/FHRS297en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS109en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS701en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS110en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS227en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS111en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS228en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS155en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS027en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS112en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS702en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS113en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS114en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS156en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS028en-GB.xml
https://ratings.food.gov.

In [ ]:
#export the selection
alldf0_99.to_csv('alldf0_99.csv')
#download the file
files.download('alldf0_99.csv')

#### The second 100

In [ ]:
#create an empty list to store the results
df_list = []

#loop through api URLs
for i in apiurl_list[100:200]:
  print(i)
  idf = parsefsaxml(i)
  #filter out exempt inspections
  idf = idf[idf['RatingValue'] != 'Exempt']
  #filter to the eating out categories
  idf = idf[
    (idf['BusinessType'] == 'Restaurant/Cafe/Canteen') |
    (idf['BusinessType'] == 'Pub/bar/nightclub') |
    (idf['BusinessType'] == 'Takeaway/sandwich shop') |
    (idf['BusinessType'] == 'Mobile caterer')
  ]
  # Append the new DataFrame to the list
  df_list.append(idf)

# Concatenate all DataFrames in the list into a single DataFrame
alldf = pd.concat(df_list)

alldf

#store this 100 in one data frame
alldf100_199 = alldf

https://ratings.food.gov.uk/api/open-data-files/FHRS521en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS522en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS523en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS524en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS525en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS526en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS527en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS528en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS529en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS530en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS531en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS532en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS533en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS874en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS706en-GB.xml
https://ratings.food.gov.

In [ ]:
#export the selection
alldf100_199.to_csv('alldf100_199.csv')
#download the file
files.download('alldf100_199.csv')

#### The third batch

In [ ]:
#create an empty list to store the results
df_list = []

#loop through api URLs
for i in apiurl_list[200:]:
  print(i)
  idf = parsefsaxml(i)
  #filter out exempt inspections
  idf = idf[idf['RatingValue'] != 'Exempt']
  #filter to the eating out categories
  idf = idf[
    (idf['BusinessType'] == 'Restaurant/Cafe/Canteen') |
    (idf['BusinessType'] == 'Pub/bar/nightclub') |
    (idf['BusinessType'] == 'Takeaway/sandwich shop') |
    (idf['BusinessType'] == 'Mobile caterer')
  ]
  # Append the new DataFrame to the list
  df_list.append(idf)

# Concatenate all DataFrames in the list into a single DataFrame
alldf = pd.concat(df_list)

alldf

#store this batch in one data frame
alldf200_ = alldf

https://ratings.food.gov.uk/api/open-data-files/FHRS106en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS310en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS140en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS187en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS885en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS270en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS877en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS311en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS312en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS189en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS313en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS142en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS190en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS191en-GB.xml
https://ratings.food.gov.uk/api/open-data-files/FHRS192en-GB.xml
https://ratings.food.gov.

In [ ]:
#export the selection
alldf200_.to_csv('alldf200_.csv')
#download the file
files.download('alldf200_.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Combine all 3 dataframes and export

Now we can combine the three dataframes we've created for the three slices of the list.

In [ ]:
#combine the 3 data frames in the list
alldf363 = pd.concat([alldf0_99,alldf100_199,alldf200_])


In [ ]:
#export as a CSV
alldf363.to_csv('alldf363.csv')
#download the file
files.download('alldf363.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Get an overview of business types

Let's check how many there are in each business type.

In [ ]:
#get a count of each type of business
alldf363['BusinessType'].value_counts()

BusinessType
Restaurant/Cafe/Canteen    132397
Takeaway/sandwich shop      63182
Pub/bar/nightclub           50997
Mobile caterer              27335
Name: count, dtype: int64

### Add the years

Each time we also extract the year of inspection into a new column.

In [ ]:
#use .to_datetime() from pandas to convert the column to datetime
#add the method .dt.year to extract the year from the resulting list of datetime objects
alldf363['ratingYear'] = pd.to_datetime(alldf363['RatingDate']).dt.year
alldf363['ratingYear'].value_counts()

ratingYear
2023.0    108549
2022.0     59659
2024.0     32732
2019.0     13963
2021.0     13637
2018.0      8102
2020.0      7153
2017.0      3710
2016.0      1944
2015.0      1238
2014.0       910
2013.0       566
2012.0       517
2011.0       481
2010.0       434
2009.0       297
2008.0       156
2007.0       102
1999.0       100
2006.0        90
2005.0        77
2004.0        61
2003.0        55
2002.0        35
2001.0        25
2000.0        13
1998.0         5
1997.0         1
1995.0         1
1996.0         1
Name: count, dtype: int64

### Add T/F columns for pre-2022

We are interested in how many haven't been inspected in at least two years. A rough approximation of that can be given by how many have a year of inspection before 2022 (there will also be some inspected in the first few months of 2022 who haven't been inspected now for over two years, but we just want a rough idea for now).



In [ ]:
alldf363['pre2022'] = alldf363['ratingYear'] < 2022
alldf363['pre2022'].value_counts()

pre2022
False    220237
True      53674
Name: count, dtype: int64

## Export with years

In [ ]:
#export as a CSV
alldf363.to_csv('alldf363.csv')
#download the file
files.download('alldf363.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Check awaiting inspection

We have a number of records where no date is given - these also return `False` for the year being before 2022.

In [ ]:
len(alldf363[alldf363['RatingValue'] == 'AwaitingInspection' ])

18650

## Generate a pivot table showing numbers for each authority.

We can get an idea of those awaiting inspection by using the `pivot_table()` function from pandas.

Note that there are two ways this is stored: as 'Awaiting Inspection' and 'AwaitingInspection' (no space).

Note also that Scottish authorities use a different rating system which has three levels: pass, pass and eat safe, and improvement required.

In [ ]:
alldf363.pivot_table(index="Authority",
                        values="BusinessName",
                        columns="RatingValue",
                        margins=True,
                        aggfunc="count").fillna(0)

RatingValue,0,1,2,3,4,5,Awaiting Inspection,AwaitingInspection,AwaitingPublication,Improvement Required,Pass,Pass and Eat Safe,All
Authority,,,,,,,,,,,,,
Aberdeen City,0.0,0.0,0.0,0.0,0.0,0.0,98.0,0.0,0.0,217.0,761.0,1.0,1077
Aberdeenshire,0.0,0.0,0.0,0.0,0.0,0.0,101.0,0.0,0.0,58.0,503.0,2.0,664
Adur,0.0,4.0,0.0,10.0,21.0,162.0,0.0,3.0,0.0,0.0,0.0,0.0,200
Amber Valley,0.0,4.0,2.0,41.0,74.0,386.0,0.0,64.0,0.0,0.0,0.0,0.0,571
Anglesey,0.0,3.0,4.0,13.0,37.0,292.0,0.0,1.0,0.0,0.0,0.0,0.0,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wychavon,0.0,4.0,6.0,32.0,81.0,303.0,0.0,49.0,0.0,0.0,0.0,0.0,475
Wyre,0.0,11.0,13.0,21.0,66.0,331.0,0.0,26.0,0.0,0.0,0.0,0.0,468
Wyre Forest,0.0,1.0,5.0,35.0,64.0,209.0,0.0,45.0,0.0,0.0,0.0,0.0,359


In [ ]:
#store in a dataframe
pivot_rating_la = alldf363.pivot_table(index="Authority",
                        values="BusinessName",
                        columns="RatingValue",
                        margins=True,
                        aggfunc="count").fillna(0)
#export as a CSV
pivot_rating_la.to_csv('pivot_rating_la.csv')

### Generate a pivot showing number of inspections pre-2022 for each authority

We can repeat this for the pre2022 column, to show the numbers in each authority which are `True` (pre-2022 inspections) and `False` (not pre-2022).

Note that those Awaiting Inspection will be counted as False here, so we need to combine the previous pivot table with this to get a more accurate figure.

In [ ]:
pivot_pre22_la = alldf363.pivot_table(index="Authority",
                        values="BusinessName",
                        columns="pre2022",
                        margins=True,
                        aggfunc="count").fillna(0).astype(int)

pivot_pre22_la


pre2022,False,True,All
Authority,,,
Aberdeen City,390,687,1077
Aberdeenshire,367,297,664
Adur,195,5,200
Amber Valley,390,181,571
Anglesey,293,57,350
...,...,...,...
Wychavon,438,37,475
Wyre,406,62,468
Wyre Forest,321,38,359


In [ ]:
#export as a CSV
pivot_pre22_la.to_csv('pivot_pre22_la.csv')

In [ ]:
#Calculate the pre22 numbers as %
pivot_pre22_la[True]/pivot_pre22_la['All']

Authority
Aberdeen City    0.637883
Aberdeenshire    0.447289
Adur             0.025000
Amber Valley     0.316988
Anglesey         0.162857
                   ...   
Wychavon         0.077895
Wyre             0.132479
Wyre Forest      0.105850
York             0.224189
All              0.195954
Length: 364, dtype: float64

In [ ]:
pivot_pre22_la['percPre22'] = pivot_pre22_la[True]/pivot_pre22_la['All']
pivot_pre22_la

pre2022,False,True,All,percPre22
Authority,,,,
Aberdeen City,390,687,1077,0.637883
Aberdeenshire,367,297,664,0.447289
Adur,195,5,200,0.025000
Amber Valley,390,181,571,0.316988
Anglesey,293,57,350,0.162857
...,...,...,...,...
Wychavon,438,37,475,0.077895
Wyre,406,62,468,0.132479
Wyre Forest,321,38,359,0.105850


## Data checking: duplicates

Let's see if there are any duplicates.

In [ ]:
#create a list of True/False values indicating whether a row is a duplicate
dupes = alldf363.duplicated()
#Get a count of T/F
dupes.value_counts()

False    273820
True         91
Name: count, dtype: int64

Is there any pattern to the authorities involved?

In [ ]:
#Use that list to filter to duplicate entries, and get a count of the authorities
alldf363['Authority'][dupes].value_counts()

Authority
Hackney                                8
Stirling                               7
Lewisham                               7
Bournemouth, Christchurch and Poole    5
Wirral                                 5
South Hams                             3
West Lothian                           3
Leeds                                  3
City of London Corporation             3
South Ribble                           3
Gateshead                              2
Bristol                                2
East Riding of Yorkshire               2
Hull City                              2
Preston                                2
Milton Keynes                          2
Kensington and Chelsea                 2
Glasgow City                           2
West Northamptonshire                  2
Renfrewshire                           2
Bromsgrove                             1
Dumfries and Galloway                  1
Highland                               1
North Lanarkshire                      1
Scotti

In [ ]:
#Use that list to filter to duplicate entries, and get a count of the authorities
alldf363['BusinessName'][dupes].value_counts()

BusinessName
W Asher And Sons                  7
Big Chief Ices                    5
The Bear and Blacksmith           2
Fotheringhams Ice Cream           2
Food 4 Festival (Greek)           2
                                 ..
Mye Green Mango                   1
Imma Kandey African Restaurant    1
Fortune House                     1
Daku Coffee One Limited           1
ESS Compass Group                 1
Name: count, Length: 76, dtype: int64

Let's just look at them.

In [ ]:
#Use that list to filter to duplicate entries, and get a count of the authorities
alldf363[dupes]

,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng,ratingYear,pre2022
70,Bildeston Baptist Church Lunch Club,Babergh,Bildeston Baptist Church,Duke Street,Bildeston,IPSWICH,IP7 7EW,5,False,2023-01-11,Restaurant/Cafe/Canteen,,,2023.0,False
401,Ongar Road Fish Bar,Brentwood,71 Ongar Road,Brentwood,Essex,,CM15 9AZ,5,False,2023-11-20,Takeaway/sandwich shop,51.6232121,0.3036676,2023.0,False
1148,Streetfood Feeding The Homeless Within The Com...,Chelmsford,,,,,,5,False,2024-03-11,Mobile caterer,,,2024.0,False
367,CornDog Row,Colchester,,,,,,AwaitingInspection,False,,Takeaway/sandwich shop,,,NaN,False
269,Hockley Cafe,Rochford,26 Spa Road,Hockley,Essex,,SS5 4PH,5,False,2019-03-18,Restaurant/Cafe/Canteen,51.601448,0.655933,2019.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,W Asher And Sons,Stirling,156 Drip Road,Raploch,Stirling,,FK8 1RR,Pass,False,2019-05-15,Mobile caterer,56.131784,-3.9563926,2019.0,True
362,Fotheringhams Ice Cream,West Lothian,Mobile,Various Locations,Trading in West Lothian,,EH54 6FF,Awaiting Inspection,False,2022-06-22,Mobile caterer,55.88695,-3.516103,2022.0,False
364,Fotheringhams Ice Cream,West Lothian,Mobile,Various Locations,Trading in West Lothian,,EH54 6FF,Awaiting Inspection,False,2022-06-22,Mobile caterer,55.88695,-3.516103,2022.0,False
645,Mr Softy,West Lothian,Mobile,Various Locations,Trading in West Lothian,,EH54 6FF,Pass,False,2017-10-17,Mobile caterer,55.88695,-3.516103,2017.0,True


## Export deduplicated

The numbers involved here are around 0.3% but we will remove them anyway.

In [ ]:
alldf363deduplicated = alldf363.drop_duplicates()
#export as a CSV
alldf363deduplicated.to_csv('alldf363deduplicated.csv')
#download the file
files.download('alldf363deduplicated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Import and deduplicate again by BusinessName/latlong

Although we have deduplicated where rows are the same, there might also be rows which are duplicates in the sense of them being the same business, but a different rating/inspection date.

Pilar Thomas conducted some spot-checking on the exported data: first, a preliminary conditional formatting in Excel to have a general view, and then using OpenRefine.

The process followed was:

1. Reorder spreadsheet so Lat column is sorted a – z, and select Reorder Rows Permanently.
2. Apply Duplicates facet to Lat and select True, so exact latitude figures will be ordered in consecutive rows.
3. Blank down cells in column BusinessName – this will identify two consecutive columns with the same BusinessName and then delete the name in the second row so it's easier to filter later. The reasoning behind this is that we'll probably then be able to detect rows with the same latitude and same business name, which could easily be duplicates.
4. Apply Facet by blank to column BusinessName.
Star these rows (732 in total) so they are easier to check later on, and then delete Facet by blank.
5. Sort A-Z by BusinessName and drag Blanks (starred rows) so they are in top of the column.
Apply Text filter to column AddressLine1.

Then random spot-checking:
6. Starting with the first row (number 1222.), search its AddressLine1 (Bickels Yard Cafe ( Fusion )) in the Text filter box.
7. Two exact rows appear with the same latitude, so we flag the starred one (with the blank BusinessName), as Bickels Yard Cafe is clearly duplicated.
8. Reset AddressLine1 Text filter and repeat this process with other random rows.

We now try to codify this process in Python.


In [ ]:
#import the exported data again if needed
exporturl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSNbsW3eA3efpYVJd_6M2Y2LttxcKPvluqmMFXnOU9_3uGnLrbwglgftFYAjgWx2PUZgMEo5q2faUCw/pub?gid=681915056&single=true&output=csv"
exportdf = pd.read_csv(exporturl)
#check how many rows
len(exportdf)

273820

In [ ]:
#remove duplicates - specifying which columns we want to deduplicate on
deduped_BNLL = exportdf.drop_duplicates(subset=['BusinessName','Lat','Lng'])
#how many does that leave
len(deduped_BNLL)

270881

### Count how many duplicates - and check

We now look at what rows have been identified as duplicates.

In [ ]:
#create a list of True/False values indicating whether a row is a duplicate
dupes = exportdf.duplicated(subset=['BusinessName','Lat','Lng'],
                            keep = False) #keep all duplicates
#Get a count of T/F
dupes.value_counts()

False    269435
True       4385
Name: count, dtype: int64

An inspection of the results flags a potential cause of false positives in the deduplication: rows where Lat and Lng are `NaN`. In these cases, only the BusinessName field is left to deduplicate on, so 14Forty in two different places is treated as a duplicate.

In [ ]:
#Use that list to filter to duplicate entries, and sort it by BusinessName so we can see them together
exportdf[dupes].sort_values(by = ['BusinessName'])

,Unnamed: 0,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng,ratingYear,pre2022
190913,1,14Forty,Herefordshire,Cider Mills,Plough Lane,NaN,Hereford,HR4 0EG,5,False,2022-03-18,Restaurant/Cafe/Canteen,NaN,NaN,2022.0,False
193859,1,14Forty,Rugby,", DHL for NHS Supply Chain Valle",Cosford Lane,NaN,Swift Valley Ind. Estate,CV21 1QN,5,False,2019-05-07,Restaurant/Cafe/Canteen,NaN,NaN,2019.0,True
142498,2,14Forty,Milton Keynes,DHL Nautilus,Pendeen Crescent,Milton Keynes,Milton Keynes,MK4 4BX,4,False,2023-05-22,Restaurant/Cafe/Canteen,NaN,NaN,2023.0,False
167813,7,23.5 Degrees Ltd T/A Starbucks,Dorset,Starbucks,NaN,Corfe Mullen,Dorset,BH21 3HA,5,False,2023-07-03,Restaurant/Cafe/Canteen,NaN,NaN,2023.0,False
167812,6,23.5 Degrees Ltd T/A Starbucks,Dorset,209,NaN,St Leonards And St Ives,Dorset,BH24 2QA,5,False,2022-07-15,Restaurant/Cafe/Canteen,NaN,NaN,2022.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143539,2334,Zorro's Cafe,Milton Keynes,"Trading Location, Central Milton Keynes Market...",NaN,NaN,NaN,NaN,AwaitingInspection,False,NaN,Restaurant/Cafe/Canteen,NaN,NaN,NaN,False
250600,6301,Zumuku Sushi,Glasgow City,NaN,1 Scotland Street,Glasgow,NaN,G5 8LS,Awaiting Inspection,False,2023-07-16,Takeaway/sandwich shop,NaN,NaN,2023.0,False
109130,5785,Zumuku Sushi,Manchester,"New Century Kitchen, New Century Hall",Mayes Street,Manchester,NaN,M60 4ES,5,False,2023-03-21,Restaurant/Cafe/Canteen,NaN,NaN,2023.0,False
170377,137,cafe2u,Exeter City,NaN,NaN,NaN,NaN,NaN,4,False,2023-06-23,Mobile caterer,NaN,NaN,2023.0,False


We can do a count to see how many there are of each Lat.

In [ ]:
#Use that list to filter to duplicate entries
#count how many of each value
exportdf['Lat'][dupes].value_counts()

Lat
53.773205    12
55.948386     8
51.304118     8
52.083159     6
53.466187     6
             ..
51.492100     2
51.361053     2
51.512870     2
51.520237     2
51.738945     2
Name: count, Length: 243, dtype: int64

### Check for `NaN` values in Lat

...but that doesn't count NaN values (it does indicate these are the minority, and that some only appear once, which shouldn't be the case if they are duplicates)

So [we need the `isnan()` function](https://www.turing.com/kb/nan-values-in-python) from the library `math`

In [ ]:
#loop through each value in Lat and use in isnan() function to return a list of True/False
#create a data frame from that list
#apply value_counts() to get a total of True and False
pd.DataFrame([math.isnan(i) for i in exportdf['Lat'][dupes]]).value_counts()

True     3843
False     542
Name: count, dtype: int64

### Deduplicating by name and postcode

Let's try doing it by postcode instead.

In [ ]:
#remove duplicates - specifying which columns we want to deduplicate on
deduped_BNP = exportdf.drop_duplicates(subset=['BusinessName','PostCode'])
#how many does that leave
len(deduped_BNP)

272192

In [ ]:
#create a list of True/False values indicating whether a row is a duplicate
dupes = exportdf.duplicated(subset=['BusinessName','PostCode'],
                            keep = False) #keep both duplicates
#Get a count of T/F
dupes.value_counts()

False    271094
True       2726
Name: count, dtype: int64

In [ ]:
#Use that list to filter to duplicate entries, and sort it by BusinessName so we can see them together
exportdf[dupes].sort_values(by = ['BusinessName'])

,Unnamed: 0,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng,ratingYear,pre2022
179252,1,A Bite To Eat,Teignbridge,Town Quay Forde Road Newton Abbot Devon,NaN,NaN,NaN,NaN,5,False,2022-11-25,Mobile caterer,50.532931,-3.596134,2022.0,False
31329,35,A Bite To Eat,Leicester City,NaN,NaN,NaN,NaN,NaN,5,False,2023-10-26,Mobile caterer,NaN,NaN,2023.0,False
177650,7,A Pizza Me,South Hams,NaN,NaN,NaN,NaN,NaN,5,False,2020-02-27,Takeaway/sandwich shop,NaN,NaN,2020.0,True
43573,4,A Pizza Me,Bexley,NaN,NaN,NaN,NaN,NaN,5,False,2023-08-21,Mobile caterer,NaN,NaN,2023.0,False
141248,14,A Touch Of Class,Medway,NaN,NaN,NaN,NaN,NaN,5,False,2022-03-18,Mobile caterer,NaN,NaN,2022.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222706,6559,Zizzi,North Yorkshire,1 Station Square,Harrogate,North Yorkshire,NaN,NaN,5,False,2023-05-19,Restaurant/Cafe/Canteen,53.992024,-1.538099,2023.0,False
143539,2334,Zorro's Cafe,Milton Keynes,"Trading Location, Central Milton Keynes Market...",NaN,NaN,NaN,NaN,AwaitingInspection,False,NaN,Restaurant/Cafe/Canteen,NaN,NaN,NaN,False
143540,2335,Zorro's Cafe,Milton Keynes,"Trading Location, Central Milton Keynes Market...",NaN,NaN,NaN,NaN,AwaitingInspection,False,NaN,Mobile caterer,NaN,NaN,NaN,False
22102,100,cafe2u,Welwyn Hatfield,NaN,NaN,NaN,NaN,NaN,5,False,2022-09-05,Mobile caterer,NaN,NaN,2022.0,False


### Check for missing PostCode field

Again, could this be caused by them having no postcode at all?

To check this we can't use `isnan()` because the column isn't numerical. So we use `isinstance()` which indicates if a variable is a specified type (in this case, `str`). An empty cell would not be a string.

In [ ]:
#check a postcode string
print(isinstance('B42 2SU', str))
#find a row with a nan value
print(exportdf['PostCode'][dupes][15])
#check that
print(isinstance(exportdf['PostCode'][dupes][15], str))

True
nan
False


In [ ]:
#loop through each value in PostCode and use in isinstance() function to return a list of True/False
#create a data frame from that list
#apply value_counts() to get a total of True and False
pd.DataFrame([isinstance(i, str) for i in exportdf['PostCode'][dupes]]).value_counts()

False    1842
True      884
Name: count, dtype: int64

So most of the duplicates here are NaN for PostCode too.

## Deduplicating only where there is not a NaN

We need to do the following:

* Sort by inspection date and BusinessName so that when we remove duplicates we remove the older inspection record
* Create a T/F column identifying duplicates based on BusinessName/Lat/Lng
* Create a T/F column identifying `NaN` entries in the Lat column
* Filter out duplicates based on BusinessName/Lat/Lng where the Lat NaN column is False
* Create a T/F column identifying duplicates based on BusinessName/PostCode
* Create a T/F column identifying `NaN` entries in the PostCode column
* Filter out duplicates based on BusinessName/PostCode where the PostCode NaN column is False
* Inspect the duplicate-but-NaN results to identify any other obvious duplicates


### Sorting by inspection date: `sort_values()`

In [ ]:
#sort by RatingDate
exportdf = exportdf.sort_values(by = ['RatingDate'])
exportdf.head(3)

,Unnamed: 0,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng,ratingYear,pre2022,duplicateBNLL,latNAN,duplicateBNP,postCodeSTR
150414,1338,Shirley And Millbrook Conservative Club,Southampton,22 Anglesea Road,Southampton,NaN,NaN,SO15 5QJ,5,False,1995-04-06,Pub/bar/nightclub,50.924719,-1.434945,1995.0,True,False,False,False,True
150673,1834,Woolston And Sholing Conservative Club,Southampton,74 St. Annes Road,Southampton,NaN,NaN,SO19 9FF,5,False,1996-06-12,Pub/bar/nightclub,50.893923,-1.368832,1996.0,True,False,False,False,True
120371,596,Corinthians FC Social Club,Westmorland and Furness,NaN,Strands Lane,NaN,Milnthorpe,LA7 7AE,3,False,1997-11-28,Pub/bar/nightclub,54.225384,-2.776882,1997.0,True,False,False,False,True


In [ ]:
#show the last rows
exportdf.tail(3)

,Unnamed: 0,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng,ratingYear,pre2022,duplicateBNLL,latNAN,duplicateBNP,postCodeSTR
273310,1199,Wild Dough Pizza,Vale of Glamorgan,NaN,NaN,NaN,NaN,NaN,AwaitingInspection,False,NaN,Mobile caterer,NaN,NaN,NaN,False,False,True,False,False
273321,1217,Y Castel,Vale of Glamorgan,"Aeros Flight Training, Cardiff International A...",NaN,Rhoose,Vale Of Glamorgan,CF62 3BD,AwaitingInspection,False,NaN,Restaurant/Cafe/Canteen,51.398575,-3.339125,NaN,False,False,False,False,True
273325,1240,YUM N' BASS,Vale of Glamorgan,NaN,NaN,NaN,NaN,NaN,AwaitingInspection,False,NaN,Mobile caterer,NaN,NaN,NaN,False,False,True,False,False


### Create True/False columns for duplicates and NaN

This time we set the `keep =` parameter to `'last'` rather than `False`. This ensures that we will only mark rows as a duplicate if they are 'older' entries (as the data is sorted by RatingDate so the last entry will be the latest).

In [ ]:
#add a T/F column identifying duplicates based on BusinessName/lat/long
exportdf['duplicateBNLL'] = exportdf.duplicated(subset=['BusinessName','Lat','Lng'],
                            keep = 'last') #Mark duplicates as True except for the last occurrence.

#show the first few rows
exportdf.head(3)

,Unnamed: 0,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng,ratingYear,pre2022,duplicateBNLL,latNAN,duplicateBNP,postCodeSTR
150414,1338,Shirley And Millbrook Conservative Club,Southampton,22 Anglesea Road,Southampton,NaN,NaN,SO15 5QJ,5,False,1995-04-06,Pub/bar/nightclub,50.924719,-1.434945,1995.0,True,False,False,False,True
150673,1834,Woolston And Sholing Conservative Club,Southampton,74 St. Annes Road,Southampton,NaN,NaN,SO19 9FF,5,False,1996-06-12,Pub/bar/nightclub,50.893923,-1.368832,1996.0,True,False,False,False,True
120371,596,Corinthians FC Social Club,Westmorland and Furness,NaN,Strands Lane,NaN,Milnthorpe,LA7 7AE,3,False,1997-11-28,Pub/bar/nightclub,54.225384,-2.776882,1997.0,True,False,False,False,True


In [ ]:
#add a T/F column identifying lat = NaN
exportdf['latNAN'] = [math.isnan(i) for i in exportdf['Lat']]

#show the first few rows
exportdf.head(3)

,Unnamed: 0,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng,ratingYear,pre2022,duplicateBNLL,latNAN,duplicateBNP,postCodeSTR
150414,1338,Shirley And Millbrook Conservative Club,Southampton,22 Anglesea Road,Southampton,NaN,NaN,SO15 5QJ,5,False,1995-04-06,Pub/bar/nightclub,50.924719,-1.434945,1995.0,True,False,False,False,True
150673,1834,Woolston And Sholing Conservative Club,Southampton,74 St. Annes Road,Southampton,NaN,NaN,SO19 9FF,5,False,1996-06-12,Pub/bar/nightclub,50.893923,-1.368832,1996.0,True,False,False,False,True
120371,596,Corinthians FC Social Club,Westmorland and Furness,NaN,Strands Lane,NaN,Milnthorpe,LA7 7AE,3,False,1997-11-28,Pub/bar/nightclub,54.225384,-2.776882,1997.0,True,False,False,False,True


In [ ]:
#add a T/F column identifying duplicates based on BusinessName/PostCode
exportdf['duplicateBNP'] = exportdf.duplicated(subset=['BusinessName','PostCode'],
                            keep = 'last') #only mark older entries as duplicates
#add a T/F column identifying PostCode = NaN
exportdf['postCodeSTR'] = [isinstance(i, str) for i in exportdf['PostCode']]

#show the first few rows
exportdf.head(3)

,Unnamed: 0,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng,ratingYear,pre2022,duplicateBNLL,latNAN,duplicateBNP,postCodeSTR
150414,1338,Shirley And Millbrook Conservative Club,Southampton,22 Anglesea Road,Southampton,NaN,NaN,SO15 5QJ,5,False,1995-04-06,Pub/bar/nightclub,50.924719,-1.434945,1995.0,True,False,False,False,True
150673,1834,Woolston And Sholing Conservative Club,Southampton,74 St. Annes Road,Southampton,NaN,NaN,SO19 9FF,5,False,1996-06-12,Pub/bar/nightclub,50.893923,-1.368832,1996.0,True,False,False,False,True
120371,596,Corinthians FC Social Club,Westmorland and Furness,NaN,Strands Lane,NaN,Milnthorpe,LA7 7AE,3,False,1997-11-28,Pub/bar/nightclub,54.225384,-2.776882,1997.0,True,False,False,False,True


### Filtering out the duplicates (but not NaN)

We can now use those T/F columns as indices to filter to the subset that we want.

For example this will use the T/F list of `postCodeSTR` to only show rows at the positions of True values (i.e. it will only show those with postcode strings):

`exportdf[exportdf['postCodeSTR']]`

We can then add another index to the end of that to filter it again: `[exportdf['duplicateBNP']` is another T/F column, where True means that row is a duplicate (but not the most recent duplicate).

In [ ]:
exportdf[exportdf['postCodeSTR']][exportdf['duplicateBNP']]

<ipython-input-79-53d9b496acb3>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exportdf[exportdf['postCodeSTR']][exportdf['duplicateBNP']]


,Unnamed: 0,BusinessName,Authority,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingValue,NewRatingPending,RatingDate,BusinessType,Lat,Lng,ratingYear,pre2022,duplicateBNLL,latNAN,duplicateBNP,postCodeSTR
236535,2157,ZULUS PERI PERI,Aberdeen City,NaN,32 Bridge Street Aberdeen,NaN,NaN,AB11 6JN,Awaiting Inspection,False,1999-04-01,Restaurant/Cafe/Canteen,57.144457,-2.101305,1999.0,True,False,False,True,True
236168,1430,PARAMOUNT,Aberdeen City,NaN,21-25 Bon Accord Street,Aberdeen,NaN,AB11 6EA,Awaiting Inspection,False,1999-04-01,Pub/bar/nightclub,57.143660,-2.105679,1999.0,True,False,False,True,True
235876,838,GOLDEN GRILL,Aberdeen City,NaN,Silverburn Crescent,Aberdeen,NaN,AB23 8EW,Awaiting Inspection,False,1999-04-01,Takeaway/sandwich shop,57.187503,-2.095280,1999.0,True,False,False,True,True
215168,1047,Carlton Social Club,Leeds,Main Street,Carlton,Rothwell,Leeds,WF3 3RW,5,False,2001-03-30,Pub/bar/nightclub,53.740684,-1.488365,2001.0,True,False,False,True,True
235864,814,GEORGE'S ICES,Aberdeen City,NaN,Stt/4/3,Registration No J930 Gck,NaN,AB23 8NT,Pass,False,2008-06-02,Mobile caterer,NaN,NaN,2008.0,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184177,2988,Fast Food Centre,Birmingham,140 Bordesley Green,Nechells,Birmingham,NaN,B9 4SU,AwaitingInspection,False,NaN,Restaurant/Cafe/Canteen,52.477075,-1.855753,NaN,False,False,False,True,True
185000,4873,Lee Garden,Birmingham,33-47 Station Road,Erdington,Birmingham,NaN,B23 6UE,AwaitingInspection,False,NaN,Takeaway/sandwich shop,52.528019,-1.840093,NaN,False,False,False,True,True
185102,5278,MAJAJAN,Birmingham,Unit 2A,Cobham Business Centre,Cobham Road,Birmingham,B9 4UP,AwaitingInspection,False,NaN,Restaurant/Cafe/Canteen,52.480626,-1.855565,NaN,False,False,False,True,True
186480,8484,Taikula chicken,Birmingham,65 Hurst Street,Birmingham,NaN,NaN,B5 4TE,AwaitingInspection,False,NaN,Restaurant/Cafe/Canteen,52.474197,-1.897146,NaN,False,False,False,True,True


There are around 461 establishments that fit this pattern. If we removed these from the data frame, how many should there be left?

In [ ]:
len(exportdf) - 461

273359

We can add a minus sign before a T/F list to reverse them (Trues become Falses and vice versa) and filter the other way, only showing records where duplicate is `False`:

`[-exportdf['duplicateBNP']`

But if we make both filters negative we don't get the 273,359 rows - we just get those which are not postcode strings and not duplicates

In [ ]:
#filter where postCodeSTR is True (the postcode is a string) and duplicate is false
#try one order of the indexing
print(len(exportdf[-exportdf['postCodeSTR']][-exportdf['duplicateBNP']]))

25198


<ipython-input-77-7e9d5945a0c4>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len(exportdf[-exportdf['postCodeSTR']][-exportdf['duplicateBNP']]))


So we need to map out what we want:

* duplicateBNP = `False`
* duplicateBNP = `True` but postCodeSTR = `False`

Let's check that if we did both and added the results together they would add up to the expected number of rows.

In [ ]:
#add the results of both filters
len(exportdf[-exportdf['duplicateBNP']]) + len(exportdf[exportdf['duplicateBNP']][-exportdf['postCodeSTR']])

<ipython-input-85-414523b58638>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(exportdf[-exportdf['duplicateBNP']]) + len(exportdf[exportdf['duplicateBNP']][-exportdf['postCodeSTR']])


273359

That adds up, so we now concatenate.

In [ ]:
#concatenate the results of both filters
deduped_BNP = pd.concat([exportdf[-exportdf['duplicateBNP']],
              exportdf[exportdf['duplicateBNP']][-exportdf['postCodeSTR']]])

len(deduped_BNP)

<ipython-input-89-22de062d078d>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exportdf[exportdf['duplicateBNP']][-exportdf['postCodeSTR']]])


273359

### Filter again to deduplicate on lat/long

This time we want:

* duplicateBNLL = `False` OR
* duplicateBNLL = `True` but latNAN = `True` (so only duplicates where there is a Lat)

We count how many fit the latter category in the dataset before we removed the duplicate postcodes:

In [ ]:
#duplicateBNLL = True AND latNAN = False
len(exportdf[exportdf['duplicateBNLL']][-exportdf['latNAN']])

<ipython-input-96-68c09d99b2f1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(exportdf[exportdf['duplicateBNLL']][-exportdf['latNAN']])


279

And how many now we've removed those postcode duplicates

In [ ]:
#duplicateBNLL = True AND latNAN = False
len(deduped_BNP[deduped_BNP['duplicateBNLL']][-deduped_BNP['latNAN']])

<ipython-input-99-95a7f6e5dd7b>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(deduped_BNP[deduped_BNP['duplicateBNLL']][-deduped_BNP['latNAN']])


18

In [ ]:
#concatenate the results of two filters
#duplicateBNLL = False
deduped_BNPLL = pd.concat([deduped_BNP[-deduped_BNP['duplicateBNLL']],
              deduped_BNP[deduped_BNP['duplicateBNLL']][deduped_BNP['latNAN']]])
              #duplicateBNLL = True AND latNAN = True, i.e. it's only a duplicate name, not location

len(deduped_BNPLL)

<ipython-input-100-1db575291be7>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  deduped_BNP[deduped_BNP['duplicateBNLL']][deduped_BNP['latNAN']]])


273341

## Export df with duplicate businesses removed

In [ ]:
deduped_BNPLL.to_csv('deduped_BNPLL.csv')
files.download('deduped_BNPLL.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Pivot deduped: inspections pre-2022 by authority

We can now repeat the analysis from earlier.

In [ ]:
pivot_pre22_la = deduped_BNPLL.pivot_table(index="Authority",
                        values="BusinessName",
                        columns="pre2022",
                        margins=True,
                        aggfunc="count").fillna(0).astype(int)

pivot_pre22_la


pre2022,False,True,All
Authority,,,
Aberdeen City,390,679,1069
Aberdeenshire,366,297,663
Adur,195,5,200
Amber Valley,390,181,571
Anglesey,293,57,350
...,...,...,...
Wychavon,437,37,474
Wyre,405,62,467
Wyre Forest,321,38,359


### Rename the columns

Let's make the column names clearer

In [ ]:
pivot_pre22_la = pivot_pre22_la.rename(columns={0:'post22',1:'pre22'})
pivot_pre22_la


pre2022,post22,pre22,All
Authority,,,
Aberdeen City,390,679,1069
Aberdeenshire,366,297,663
Adur,195,5,200
Amber Valley,390,181,571
Anglesey,293,57,350
...,...,...,...
Wychavon,437,37,474
Wyre,405,62,467
Wyre Forest,321,38,359


### Pivot deduped: rating by authority

In [ ]:
#store in a dataframe
pivot_rating_la = deduped_BNPLL.pivot_table(index="Authority",
                        values="BusinessName",
                        columns="RatingValue",
                        margins=True,
                        aggfunc="count").fillna(0)
pivot_rating_la

RatingValue,0,1,2,3,4,5,Awaiting Inspection,AwaitingInspection,AwaitingPublication,Improvement Required,Pass,Pass and Eat Safe,All
Authority,,,,,,,,,,,,,
Aberdeen City,0.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,217.0,756.0,1.0,1069
Aberdeenshire,0.0,0.0,0.0,0.0,0.0,0.0,101.0,0.0,0.0,58.0,502.0,2.0,663
Adur,0.0,4.0,0.0,10.0,21.0,162.0,0.0,3.0,0.0,0.0,0.0,0.0,200
Amber Valley,0.0,4.0,2.0,41.0,74.0,386.0,0.0,64.0,0.0,0.0,0.0,0.0,571
Anglesey,0.0,3.0,4.0,13.0,37.0,292.0,0.0,1.0,0.0,0.0,0.0,0.0,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wychavon,0.0,4.0,6.0,31.0,81.0,303.0,0.0,49.0,0.0,0.0,0.0,0.0,474
Wyre,0.0,11.0,13.0,21.0,66.0,331.0,0.0,25.0,0.0,0.0,0.0,0.0,467
Wyre Forest,0.0,1.0,5.0,35.0,64.0,209.0,0.0,45.0,0.0,0.0,0.0,0.0,359


### Export

In [ ]:
#export as a CSV
pivot_rating_la.to_csv('pivot_rating_la.csv')
pivot_pre22_la.to_csv('pivot_pre22_la.csv')
#download
files.download('pivot_pre22_la.csv')
files.download('pivot_rating_la.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Some previous data checking

There were some duplicates in a previous run of this notebook. This is how we saw if there's a pattern before we removed them.

In [ ]:
#import the exported data
exporturl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQCqMuFBH7EA88vdfYXhoQ42wUT-BIlVYyMhJmIkwSYaXvEE6A4IKN_WM3cztF2tFVUIx0ImZDo63RE/pub?gid=2131543290&single=true&output=csv"
exportdf = pd.read_csv(exporturl)
#check how many rows
len(exportdf)

272106

In [ ]:
#remove duplicates
deduped = exportdf.drop_duplicates()
#how many does that leave
len(deduped)

257783

In [ ]:
#create a list of True/False values indicating whether a row is a duplicate
dupes = exportdf.duplicated()
#Get a count of T/F
dupes.value_counts()

False    257783
True      14323
Name: count, dtype: int64

In [ ]:
#Use that list to filter to duplicate entries, and get a count of the authorities
exportdf['Authority'][dupes].value_counts()

Authority
Cardiff              1463
Swansea              1083
Powys                 978
Rhondda Cynon Taf     943
Pembrokeshire         911
Carmarthenshire       902
Gwynedd               900
Conwy                 729
Caerphilly            619
Neath Port Talbot     604
Newport               578
Bridgend              562
Vale of Glamorgan     523
Flintshire            514
Wrexham               491
Denbighshire          464
Monmouthshire         447
Ceredigion            399
Anglesey              350
Torfaen               323
Blaenau Gwent         272
Merthyr Tydfil        268
Name: count, dtype: int64

In [ ]:
len(alldf.drop_duplicates())

2699